In [1]:
import json
import numpy as np
from game2048.game import Game

In [6]:
from collections import Counter
from exp_imitation_learning.model import RNN
from game2048.agents import ExpectiMaxAgent as TestAgent


In [12]:
import json
import numpy as np
from game2048.game import Game


def generate_fingerprint(AgentClass, **kwargs):
    with open("board_cases.json") as f:
        board_json = json.load(f)

    game = Game(size=4, enable_rewrite_board=True)
    agent = AgentClass(game=game, **kwargs)

    trace = []
    for board in board_json:
        game.board = np.array(board)
        direction = agent.step()
        trace.append(direction)
    fingerprint = "".join(str(i) for i in trace)
    return fingerprint


if __name__ == '__main__':
    from collections import Counter

    '''====================
    Use your own agent here.'''
    from exp_imitation_learning.model import RNN
    from game2048.agents import ExpectiMaxAgent as TestAgent
    '''===================='''

    fingerprint = generate_fingerprint(TestAgent)

    with open("EE369_fingerprint.json", 'w') as f:        
        pack = dict()
        pack['fingerprint'] = fingerprint
        pack['statstics'] = dict(Counter(fingerprint))
        f.write(json.dumps(pack, indent=4))

In [9]:
def generate_fingerprint(AgentClass, **kwargs):
    i=0
    with open("board_cases.json") as f:
        board_json = json.load(f)

    game = Game(size=4, enable_rewrite_board=True)
    agent = AgentClass(game=game, **kwargs)

    trace = []
    for board in board_json:
        game.board = np.array(board)
        direction = agent.step()
        trace.append(direction)
        i=i+1
        if i==1:
            break
            
    fingerprint = "".join(str(i) for i in trace)
    
    
    return fingerprint

In [10]:
fingerprint = generate_fingerprint(TestAgent)

In [1]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
import game2048.agents
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent, getBoardFormExpect, MyRnnAgent
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import time
from torch.autograd import Variable
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [184]:
#参数设置
NUM_EPOCHS = 100
BATCH_SIZE = 64
TIME_STEP = 4
INPUT_SIZE = 4
LR = 0.001
test_size = 4
trainfilertoread = '/DATA5_DB8/data/gjliu/game2048/dataset/Train2.csv'
testfiletoread = 'Test.csv'

In [5]:
display1 = Display()
display2 = IPythonDisplay()

In [ ]:
#生成数据集
for i in range(5000):
    game = Game(4, score_to_win=2048, random=False)
    display2.display(game)
    agent2 = getBoardFormExpect(game, display=display2)
    agent2.writeBoard()
    print(i)
for i in range(17000):
    game = Game(4, score_to_win=1024, random=False)
    display2.display(game)
    agent2 = getBoardFormExpect(game, display=display2)
    agent2.writeBoard()
    print(i)

,,,
,,,
,,,
4,,4,


0


,,,
,,,
,,,
,4,,2


1


,,,
,2,,
,,2,
,,,


...

,,,
,,,2
,,2,
,,,


60


,2,,
,,,
,,,
,,4,


61


,,,
4,,,
,,,
4,,,


In [189]:
trainDataset = DealDataset(root=trainfilertoread, transform=transforms.Compose(transforms=[transforms.ToTensor()]))
train_loader = DataLoader(dataset=trainDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [191]:
#model = RNN()
model = torch.load('/DATA5_DB8/data/gjliu/game2048/model/model_rnn_256_dataup/rnn_256_model_24.pkl', map_location='cpu')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0003)
print(LR)

0.001


In [ ]:
for epoch in range(10):
    
    for index, (board, direc) in enumerate(train_loader):
        board, direc = Variable(board), Variable(direc)
        if torch.cuda.is_available():
            board, direc = board.cuda(), direc.cuda()
            model.cuda()

        board = board.view(-1,4,4)
        out = model(board)
        loss = criterion(out, direc)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # train_loss += loss.data[0]
        # pred = torch.max(out, 1)[1]
        # train_correct = (pred == direc).sum().item()
        # train_acc += train_correct

        if index % 50 == 0:
            out = model(board)
            pred = torch.max(out, 1)[1]
            train_correct = (pred == direc).sum().item()
            print('Epoch: ', epoch, '| train loss: %.4f' % loss,
                  '| test accuracy: %.4f' % (train_correct/(BATCH_SIZE * 1.0)))
    torch.save(model, '/DATA5_DB8/data/gjliu/game2048/model/model_rnn_256_dataup/rnn_256_model_' + str(epoch+25) + '.pkl')
torch.save(model, '/DATA5_DB8/data/gjliu/game2048/model/model_rnn_256_dataup/rnn_256_model_final.pkl')

Epoch:  0 | train loss: 0.6335 | test accuracy: 0.7344
Epoch:  0 | train loss: 0.3647 | test accuracy: 0.8438
Epoch:  0 | train loss: 0.3367 | test accuracy: 0.8281
Epoch:  0 | train loss: 0.4926 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.5435 | test accuracy: 0.7656
Epoch:  0 | train loss: 0.3820 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.3608 | test accuracy: 0.8281
Epoch:  0 | train loss: 0.5621 | test accuracy: 0.7812
Epoch:  0 | train loss: 0.3295 | test accuracy: 0.8594
Epoch:  0 | train loss: 0.4697 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.4060 | test accuracy: 0.8281
Epoch:  0 | train loss: 0.3706 | test accuracy: 0.8438
Epoch:  0 | train loss: 0.5368 | test accuracy: 0.7344
Epoch:  0 | train loss: 0.4354 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.4772 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.5310 | test accuracy: 0.7344
Epoch:  0 | train loss: 0.6547 | test accuracy: 0.7656
Epoch:  0 | train loss: 0.5220 | test accuracy: 0.7656
Epoch:  0 

Epoch:  0 | train loss: 0.4638 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.3592 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.5494 | test accuracy: 0.7500
Epoch:  0 | train loss: 0.4624 | test accuracy: 0.8281
Epoch:  0 | train loss: 0.5246 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.5815 | test accuracy: 0.7188
Epoch:  0 | train loss: 0.3892 | test accuracy: 0.8438
Epoch:  0 | train loss: 0.4927 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.4694 | test accuracy: 0.7812
Epoch:  0 | train loss: 0.4902 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.4769 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.5180 | test accuracy: 0.7656
Epoch:  0 | train loss: 0.5468 | test accuracy: 0.7656
Epoch:  0 | train loss: 0.6086 | test accuracy: 0.6875
Epoch:  0 | train loss: 0.5145 | test accuracy: 0.7188
Epoch:  0 | train loss: 0.3924 | test accuracy: 0.8438
Epoch:  0 | train loss: 0.6157 | test accuracy: 0.7344
Epoch:  0 | train loss: 0.4885 | test accuracy: 0.7188
Epoch:  0 

Epoch:  0 | train loss: 0.4830 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.5706 | test accuracy: 0.7500
Epoch:  0 | train loss: 0.5117 | test accuracy: 0.6875
Epoch:  0 | train loss: 0.3984 | test accuracy: 0.8281
Epoch:  0 | train loss: 0.4327 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.4561 | test accuracy: 0.7656
Epoch:  0 | train loss: 0.5740 | test accuracy: 0.6719
Epoch:  0 | train loss: 0.3407 | test accuracy: 0.8906
Epoch:  0 | train loss: 0.3258 | test accuracy: 0.8750
Epoch:  0 | train loss: 0.4653 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.5959 | test accuracy: 0.7344
Epoch:  0 | train loss: 0.5972 | test accuracy: 0.7188
Epoch:  0 | train loss: 0.4248 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.4184 | test accuracy: 0.7812
Epoch:  0 | train loss: 0.5259 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.4082 | test accuracy: 0.8281
Epoch:  0 | train loss: 0.5118 | test accuracy: 0.7188
Epoch:  0 | train loss: 0.4128 | test accuracy: 0.8125
Epoch:  0 

Epoch:  0 | train loss: 0.5988 | test accuracy: 0.7344
Epoch:  0 | train loss: 0.6125 | test accuracy: 0.7031
Epoch:  0 | train loss: 0.4309 | test accuracy: 0.7500
Epoch:  0 | train loss: 0.6272 | test accuracy: 0.7188
Epoch:  0 | train loss: 0.3885 | test accuracy: 0.8438
Epoch:  0 | train loss: 0.4066 | test accuracy: 0.8281
Epoch:  0 | train loss: 0.4417 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.4083 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.5151 | test accuracy: 0.7344
Epoch:  0 | train loss: 0.5128 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.5361 | test accuracy: 0.7969
Epoch:  0 | train loss: 0.4936 | test accuracy: 0.7812
Epoch:  0 | train loss: 0.5287 | test accuracy: 0.7344
Epoch:  0 | train loss: 0.6851 | test accuracy: 0.6406
Epoch:  0 | train loss: 0.6872 | test accuracy: 0.8125
Epoch:  0 | train loss: 0.6124 | test accuracy: 0.7656
Epoch:  0 | train loss: 0.4389 | test accuracy: 0.7500
Epoch:  0 | train loss: 0.6125 | test accuracy: 0.7812
Epoch:  0 

In [ ]:
game = Game(4, score_to_win=2048, random=False)
game2 =game
display2.display(game)
# my agent
agent = MyRnnAgent(game, display=display2)
agent.play(verbose=True)

In [74]:
agent2 = MyRnnAgent2(game2, display=display2)
agent2.play(verbose=True)

In [75]:
print(game2.__board)

AttributeError: 'Game' object has no attribute '__board'

In [30]:
game = Game(4, score_to_win=2048, random=False)
display2.display(game)
agent = ExpectiMaxAgent(game, display=display2)
agent.play(verbose=True)

tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0909, 0.0000],
         [0.0000, 0.1818, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]]) 3


In [8]:
Data0 = pd.read_csv('/DB/rhome/gjliu/ML-EE228/2048-api/Train.csv').values

In [9]:
print(np.shape(Data0))
print(type(Data0))
print((Data0[0, 0:-1]))
print((Data0[0, :]))
print(Data0[0, -1])

(696, 17)
<class 'numpy.ndarray'>
[0 0 0 0 0 0 1 0 0 2 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 2 0 0 0 0 0 0 3]
3


In [26]:
board = Data0[:, 0:-1]
board = board[0].reshape((4, 4))
print(board)
print(board.shape)
board = board[:, :, np.newaxis]
board = board/11.0
#print(board)
print(board.shape)
direc = Data0[:, -1]
print(direc[7])
board = transforms.Compose(transforms=[transforms.ToTensor()])(board)
print(board)
print(type(board))
print(board.shape)
board = board.type(torch.float)
print(board)
print(type(board))
print(board.shape)
print(len(Data0))

[[0 0 0 0]
 [0 0 1 0]
 [0 2 0 0]
 [0 0 0 0]]
(4, 4)
(4, 4, 1)
3
tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0909, 0.0000],
         [0.0000, 0.1818, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]], dtype=torch.float64)
<class 'torch.Tensor'>
torch.Size([1, 4, 4])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0909, 0.0000],
         [0.0000, 0.1818, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])
<class 'torch.Tensor'>
torch.Size([1, 4, 4])
696


In [11]:
game = Game(4, score_to_win=2048, random=False)
display2.display(game)
print(game.board)
board = np.where(game.board == 0, 1, game.board)
print(board)
board = np.log2(board)
# print(board)
# board = board.reshape((4, 4))
print(board)
board = board[:, :, np.newaxis]
print(board)

,,,
,2,,
4,,,
,,,


[[0. 0. 0. 0.]
 [0. 2. 0. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [4. 1. 1. 1.]
 [1. 1. 1. 1.]]
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[[0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[2.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]]]


In [64]:
model = torch.load('/DB/rhome/gjliu/ML-EE228/2048-api/model/rnn_model_final.pkl', map_location='cpu')
game = Game(4, score_to_win=2048, random=True)
board = np.where(game.board == 0, 1, game.board)
board = np.log2(board)
board = board[:, :, np.newaxis]

board = board/ 11.0

trans = transforms.Compose([transforms.ToTensor()])
board = trans(board)

board = board.type(torch.float)

out = model(board)

direction = torch.max(out, 1)[1]
print(direction)

tensor([1])


In [36]:
def onehot(board):
    ret = np.zeros(shape=(4,4,16),dtype=bool)
    for r in range(4):
        for c in range(4):
            ret[r,c,board[r,c]] = 1
    return ret


Data0 = pd.read_csv('/DATA5_DB8/data/gjliu/game2048/dataset/Train_special.csv').values
board = Data0[:, 0:-1]
board = board[1].reshape((4, 4))
print(board)
board = onehot(board)
board = board.astype(np.int)

print(board)


[[0 0 0 1]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]]
